In [1]:
device = 'cuda:3'

In [2]:
from data_loader import get_mix_instruct

prompts, references, ds_name = get_mix_instruct("train", 21000)

Dataset: mix-instruct_train_21000 found in cache, loading from cache ✅


In [3]:
from utility_functions.delift_se import get_delift_se_utility

utility, utility_name = get_delift_se_utility(prompts, references, ds_name)

Utility: mix-instruct_train_21000_delift-se found in cache, loading from cache ✅


In [5]:
from subset import create_subset, get_subset

subset, subset_name = create_subset(utility, utility_name)

Subset: mix-instruct_train_21000_delift-se_0.3 found in cache, loading from cache ✅


In [5]:
from inference import generate_responses
from evaluation import compute_metrics
# For LLama 3.2 (initial)
# Reported BGE: 0.73, ROUGE: 0.28
responses_llama, generation_name_llama = generate_responses(prompts, 'meta-llama/Llama-3.2-3B', ds_name, device, batch_size=400)
compute_metrics(responses_llama, references, generation_name_llama, device=device)

Generation: mix-instruct_train_21000_Llama-3.2-3B found in cache, loading from cache ✅
Evaluate: mix-instruct_train_21000_Llama-3.2-3B_bge found in cache, loading from cache ✅
Evaluate: mix-instruct_train_21000_Llama-3.2-3B_rouge found in cache, loading from cache ✅


{'bge': 0.712607, 'rouge': 0.21774006690983283}

In [7]:
# For LLama 3.2 (finetuned on 30% data selected with DeLiftSE)
# Reported BGE: 0.85, ROUGE: 0.55
from finetune import fine_tune_model
from data_loader import get_mix_instruct
from utility_functions.delift_se import get_delift_se_utility
from subset import create_subset, get_subset
import importlib
import inference
importlib.reload(inference)
from inference import generate_responses
from evaluation import compute_metrics



prompts, references, ds_name = get_mix_instruct("train", 21000)
utility, utility_name = get_delift_se_utility(prompts, references, ds_name)
subset, subset_name = create_subset(utility, utility_name)
s_prompts, s_references = get_subset(subset, prompts, references)
prompts_val, references_val, ds_name_valid = get_mix_instruct("validation", 5000)
model_dir = fine_tune_model('meta-llama/Llama-3.2-3B', prompts, references, prompts_val, references_val, subset_name)

responses_llama_ft, generation_name_llama_ft = generate_responses(prompts_val, model_dir, ds_name_valid, device, batch_size=400)
compute_metrics(responses_llama_ft, references_val, generation_name_llama_ft, device=device)

Dataset: mix-instruct_train_21000 found in cache, loading from cache ✅
Utility: mix-instruct_train_21000_delift-se found in cache, loading from cache ✅
Subset: mix-instruct_train_21000_delift-se_0.3 found in cache, loading from cache ✅
Dataset: mix-instruct_validation_5000 found in cache, loading from cache ✅
Finetune: Fine-tuned model found in cache. Skipping Training ✅
Generation: mix-instruct_validation_5000_Llama-3.2-3B_mix-instruct_train_21000_delift-se_0.3_100 found in cache, loading from cache ✅
Evaluate: mix-instruct_validation_5000_Llama-3.2-3B_mix-instruct_train_21000_delift-se_0.3_100_bge found in cache, loading from cache ✅
Evaluate: mix-instruct_validation_5000_Llama-3.2-3B_mix-instruct_train_21000_delift-se_0.3_100_rouge found in cache, loading from cache ✅


{'bge': 0.75713426, 'rouge': 0.3198248393101128}